In [29]:
import numpy as np
import pandas as pd
from scipy.stats import wilcoxon
from scipy.stats import ttest_1samp


In [37]:
folderpath = "../../Data/QMEF/"
btc_tweets = pd.read_csv('BTC_Tweets.csv')
btc = pd.read_csv(folderpath + 'BTC-USD.csv') # Datum aufsteigend

doge_tweets = pd.read_csv('Doge_Tweets.csv')
doge = pd.read_csv(folderpath + 'DOGE-USD.csv') # Datum aufsteigend

cci30 = pd.read_csv(folderpath + 'cci30_OHLCV.csv') # Datum absteigend!
cci30 = cci30.sort_values(by=['Date'], ascending=True) # Datum aufsteigend!
cci30

,Date,Open,High,Low,Close,Volume
3161,2015-01-01,101.3600,102.1530,98.8187,100.0000,0.000000e+00
3160,2015-01-02,99.9137,101.8690,98.7938,100.2180,0.000000e+00
3159,2015-01-03,100.2530,101.5400,88.7442,89.3769,0.000000e+00
3158,2015-01-04,89.2479,91.1656,79.8710,82.2132,0.000000e+00
3157,2015-01-05,82.3484,88.3190,81.5787,86.2289,0.000000e+00
...,...,...,...,...,...,...
4,2023-08-24,7286.9100,7330.1200,7103.4200,7181.7400,2.229648e+10
3,2023-08-25,7182.1500,7230.5500,7049.1700,7150.2300,1.962683e+10
2,2023-08-26,7150.2200,7186.4600,7103.3400,7141.9300,1.111184e+10
1,2023-08-27,7141.8900,7227.2000,7118.5300,7188.4000,1.579862e+10


In [38]:
cci30['Date'] = pd.to_datetime(cci30['Date'])
btc['Date'] = pd.to_datetime(btc['Date'])
doge['Date'] = pd.to_datetime(doge['Date'])

cci30['Return'] = (cci30['Close']/cci30['Close'].shift(1))
btc['Return'] = (btc['Close']/btc['Close'].shift(1))
doge['Return'] = (doge['Close']/doge['Close'].shift(1))

In [39]:
def get_bhar(tweets, values, cci30):
    BHAR = []
    for d in pd.to_datetime(tweets['UTC'].str[:10]):
        sliced_vals = values[(values['Date'] >= d) & ((values['Date'] <= d + pd.Timedelta(days=28)))]
        cci30_sliced = cci30[(cci30['Date'] >= d) & ((cci30['Date'] <= d + pd.Timedelta(days=28)))]
        
        e1 = np.prod(sliced_vals['Return'])
        e2 = np.prod(cci30_sliced['Return'])

        BHAR.append(e1 - e2)

    return BHAR

bhar_btc = (get_bhar(btc_tweets, btc, cci30))
bhar_doge = (get_bhar(doge_tweets, doge, cci30))

print(bhar_btc)
print(bhar_doge)

[0.047629030746414225, -0.06303059030617608, -0.1346292920801888, -0.07587504249110355, -0.22146360463574677, 0.05142948162623828]
[0.07354535495339642, -0.2969398586122505, -0.13022540222249668, -0.174684993525551, -0.6912743283611119, -0.5037741073903583, 0.4975937340049511, -0.11229578862020362, 0.11887944833007502, -0.15337447330034681, -0.15337447330034681, 0.17060732388715272, 1.6846604158026366, -0.5258460022887418, 4.381683790888376, -0.034885462912705445, 0.03471714541413151, -0.524561146274419, 0.8958725939196419]


In [45]:
bhar_total = bhar_doge + bhar_btc
print(len(bhar_total))
print(np.mean(bhar_doge))
print(np.mean(bhar_btc))
print(np.mean(bhar_total))

25
0.2398065142311489
-0.06599000285676045
0.16641535013005065


In [46]:
# Annahme: data enthält deine Stichprobe von log-normalverteilten Daten
def bootstrap(data):
    num_bootstrap_samples = 10000  # Anzahl der Bootstrap-Stichproben

    bootstrap_means = []
    for _ in range(num_bootstrap_samples):
        bootstrap_sample = np.random.choice(data, size=len(data), replace=True)
        bootstrap_mean = np.mean(bootstrap_sample)
        bootstrap_means.append(bootstrap_mean)

    # Berechne das Konfidenzintervall
    print('Wilcoxon p-Value', wilcoxon(data, method='exact', alternative='two-sided')[1])
    print(ttest_1samp(data, 0, alternative='two-sided'))
    print("90%-Konfidenzintervall:", np.percentile(bootstrap_means, [5, 95]))
    print("95%-Konfidenzintervall:", np.percentile(bootstrap_means, [2.5, 97.5]))
    print("99%-Konfidenzintervall:", np.percentile(bootstrap_means, [0.5, 99.5]), '\n')
    

print('Doge:')
bootstrap(bhar_doge)

print('BTC')
bootstrap(bhar_btc)

print('Total')
bootstrap(bhar_total)


Doge:
Wilcoxon p-Value 84.0
TtestResult(statistic=0.9143531225679291, pvalue=0.37261770891518964, df=18)
90%-Konfidenzintervall: [-0.12573507  0.69177968]
95%-Konfidenzintervall: [-0.16556364  0.78986473]
99%-Konfidenzintervall: [-0.2284119   0.99593328] 

BTC
Wilcoxon p-Value 3.0
TtestResult(statistic=-1.531665291929547, pvalue=0.1861712978705852, df=5)
90%-Konfidenzintervall: [-0.1320562  -0.00206455]
95%-Konfidenzintervall: [-0.14398931  0.00986857]
99%-Konfidenzintervall: [-0.16825407  0.02981917] 

Total
Wilcoxon p-Value 127.0
TtestResult(statistic=0.8319626138742418, pvalue=0.41363188112341387, df=24)
90%-Konfidenzintervall: [-0.11508361  0.52370634]
95%-Konfidenzintervall: [-0.14537115  0.61030628]
99%-Konfidenzintervall: [-0.18884924  0.77270613] 

